In [ ]:
# Importation des bibliothèques nécessaires
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Étape 1 : Chargement et prétraitement des données
# Le dataset Fashion MNIST contient 70 000 images de vêtements en 10 catégories.

# Charger les données Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Les images sont en niveaux de gris (28x28). Ajoutons une dimension pour correspondre au format attendu par CNN (28x28x1).
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0  # Normalisation entre 0 et 1
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Encodage des labels en one-hot (exemple : 3 -> [0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Étape 2 : Définition du modèle CNN
# On utilise le modèle séquentiel pour empiler les couches.

def create_cnn_model():
    model = Sequential()

    # Première couche convolutionnelle
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))  # Réduction de la taille de l'image

    # Deuxième couche convolutionnelle
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Aplatissement des caractéristiques (passage de 2D à 1D)
    model.add(Flatten())

    # Couche dense (fully connected) pour l'apprentissage
    model.add(Dense(128, activation='relu'))  # 128 neurones
    model.add(Dropout(0.5))  # Régularisation pour éviter l'overfitting

    # Couche de sortie avec softmax pour la classification en 10 classes
    model.add(Dense(10, activation='softmax'))

    # Compilation du modèle
    model.compile(
        optimizer='adam',  # Optimiseur Adam
        loss='categorical_crossentropy',  # Fonction de perte pour la classification multi-classes
        metrics=['accuracy']  # Métrique pour évaluer les performances
    )

    return model

# Créer le modèle
model = create_cnn_model()

# Résumé du modèle
model.summary()

# Étape 3 : Entraînement du modèle
# On entraîne le modèle sur les données d'entraînement et on évalue sur les données de test.

history = model.fit(
    x_train, y_train,  # Données d'entraînement
    batch_size=64,  # Nombre d'échantillons par batch
    epochs=10,  # Nombre d'époques (itérations complètes sur les données)
    validation_split=0.2,  # 20% des données d'entraînement utilisées pour la validation
    verbose=1  # Affichage détaillé
)

# Étape 4 : Évaluation sur les données de test
# Cela permet de voir la précision du modèle sur des données jamais vues.

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Précision sur les données de test : {test_accuracy:.2f}")

# Étape 5 : Visualisation des courbes d'entraînement (optionnel)
import matplotlib.pyplot as plt

# Courbe de perte
plt.plot(history.history['loss'], label='Perte d\'entraînement')
plt.plot(history.history['val_loss'], label='Perte de validation')
plt.title('Courbe de Perte')
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.legend()
plt.show()

# Courbe de précision
plt.plot(history.history['accuracy'], label='Précision d\'entraînement')
plt.plot(history.history['val_accuracy'], label='Précision de validation')
plt.title('Courbe de Précision')
plt.xlabel('Époques')
plt.ylabel('Précision')
plt.legend()
plt.show()


In [ ]:
# Install Keras Tuner if not already installed (uncomment if needed)
!pip install keras-tuner --quiet

# Import Keras Tuner
import keras_tuner as kt

# Redefine the model function to allow hyperparameter tuning
def build_cnn_model(hp):
    model = Sequential()

    # First convolutional layer with tunable filters
    model.add(Conv2D(
        filters=hp.Int('conv_1_filters', min_value=16, max_value=64, step=16),  # Tune filters
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(28, 28, 1)
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second convolutional layer with tunable filters
    model.add(Conv2D(
        filters=hp.Int('conv_2_filters', min_value=32, max_value=128, step=32),  # Tune filters
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten layer
    model.add(Flatten())

    # Dense layer with tunable units
    model.add(Dense(
        units=hp.Int('dense_units', min_value=64, max_value=256, step=64),  # Tune dense units
        activation='relu'
    ))
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))  # Tune dropout rate

    # Output layer
    model.add(Dense(10, activation='softmax'))

    # Compile the model with a tunable optimizer
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),  # Tune optimizer
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Instantiate the tuner
tuner = kt.Hyperband(
    build_cnn_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='tuner_results',
    project_name='fashion_mnist_cnn'
)

# Search for the best hyperparameters
tuner.search(
    x_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print(f"""
Optimal number of filters for the first convolutional layer: {best_hps.get('conv_1_filters')}
Optimal number of filters for the second convolutional layer: {best_hps.get('conv_2_filters')}
Optimal number of dense units: {best_hps.get('dense_units')}
Optimal dropout rate: {best_hps.get('dropout_rate')}
Optimal optimizer: {best_hps.get('optimizer')}
""")

# Train the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)

# Evaluate the best model on test data
test_loss, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)
print(f"Précision sur les données de test avec le modèle optimisé : {test_accuracy:.2f}")


**Travail à faire:** Optimiser les hyperparamètres (taux d’apprentissage, nombre de filtres, taille des noyaux, etc.) à l’aide de Grid Search, Random Search ou Keras Tuner.
Comparer les performances avec des modèles optimisés.